# 03 - Member's opt-in and receive voting tokens

In this notebook we need to implement a token-based membership system where each senator with the right to vote opts in to the voting assets to be able to vote. Each member receives 1 USIV token for each voting proposal so that it can express a vote using the ASA token so to make sure that the voting happens only for members that are inside the association (if they are inside the association, then it must be the case that they received the token already)

### 0. Load the required assets

In [20]:
# Add "algo_util.py" to the list of available dependencies of the project
import sys, os, json
from pyteal import *
from pathlib import Path

codepath = (Path( os.curdir ) / "shared").resolve()
sys.path.append(str(codepath))

In [21]:
# import some utilities
from algo_util import wait_for_confirmation, load_credentials
from algo_util import format_state, read_global_state, read_local_state

In [22]:
# algo-sdk utilities
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn, AssetOptInTxn
import algosdk.error
import json
import base64
import datetime

In [23]:
# loading credential utility
from algo_util import load_credentials 

# load credentials from storage
credentials = load_credentials(file_name = "credentials_temp")

print(f"{len(credentials['members_credentials'])} members' credentials loaded from storage")
print(f"Loading also USIV account credentials: {credentials['usiv']['public']}")

32 members' credentials loaded from storage
Loading also USIV account credentials: FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI


Initialize the algorand client to interact with the testnet

In [24]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(
    algod_token='', 
    algod_address=credentials['algod_test'], 
    headers=credentials['purestake_token']
)

Retrieve the information on the asset to be used to vote

In [25]:
# loading credential utility
from algo_util import load_credentials 

# load credentials from storage
credentials = load_credentials(file_name = "credentials_temp")

print(f"{len(credentials['members_credentials'])} members' credentials loaded from storage")
print(f"Loading also USIV account credentials: {credentials['usiv']['public']}")

32 members' credentials loaded from storage
Loading also USIV account credentials: FQYVLYY73LY723WD2462VDSM4UWA4FYA477V4E277JDQCDCN326QBCDPUI


In [30]:
# assuming the USIV account created only 1 coin
coin_id = algod_client.account_info(
    credentials['usiv']['public'])["assets"][1]["asset-id"]
print(f"The asset (coin) id is: {coin_id}")

The asset (coin) id is: 684625427


### 1. Opt-in the members

**Opt-in** all the accounts that need to express a vote

In [31]:
def optin_asset_member(member_pubk, member_prik, coin_id):
    """
        Opt-in to the asset for voting
    """
    sp = algod_client.suggested_params()
    optin_txn = AssetOptInTxn(sender=member_pubk,
                              sp=sp,
                              index=coin_id)
    signed_txn = optin_txn.sign(member_prik)
    txid = algod_client.send_transaction(signed_txn)
    wait_for_confirmation(algod_client,txid)

In [32]:
# asset opt-int configurations
sp                  = algod_client.suggested_params()
idx_coin            = coin_id
usiv                = credentials['usiv']

In [33]:
# loop through all members to opt them in
for i, member in enumerate(credentials["members_credentials"]):
    # retrieve member's keys
    member_pubk = credentials["members_credentials"][member]["public"]
    member_prik = credentials["members_credentials"][member]["private"]

    # # # debugging purposes
    # if i == 7:
    #     break

    # opt-in in case he did not do it yet
    if algod_client.account_info(member_pubk)["assets"]:
        if algod_client.account_info(member_pubk)["assets"][1]["asset-id"] == coin_id:
            print(f"{member} has already opted-in to coin '{algod_client.asset_info(idx_coin)['params']['unit-name']}'")
            continue

    # opt-in transaction
    optin_asset_member(member_pubk, member_prik, idx_coin)

    # result of the opt-in operation
    print(f"{member} has opted-in to coin {algod_client.asset_info(idx_coin)['params']['unit-name']}")


Murphy has already opted-in to coin 'USIV'
Shaffer has already opted-in to coin 'USIV'
Beck has already opted-in to coin 'USIV'
Smith has already opted-in to coin 'USIV'
Gibson has already opted-in to coin 'USIV'
Duncan has already opted-in to coin 'USIV'
Powers has already opted-in to coin 'USIV'
Jordan has already opted-in to coin 'USIV'
Atkins has already opted-in to coin 'USIV'
Carr has already opted-in to coin 'USIV'
Pacheco has already opted-in to coin 'USIV'
Hanson has already opted-in to coin 'USIV'
Diaz has already opted-in to coin 'USIV'
Williams has already opted-in to coin 'USIV'
Wright has already opted-in to coin 'USIV'
Harris has already opted-in to coin 'USIV'
Valdez has already opted-in to coin 'USIV'
Davidson has already opted-in to coin 'USIV'
Stewart has already opted-in to coin 'USIV'
Bell has already opted-in to coin 'USIV'
MD has already opted-in to coin 'USIV'
Moore has already opted-in to coin 'USIV'
Thomas has already opted-in to coin 'USIV'
Jackson has alread

### 2. Transfer coins to the members

**Transfer 1 coin** to all the members that needs to express a vote

In [34]:
def asset_transfer_txn(usiv, member_pubk, idx_coin, amount_for_voting):
    """
        Transfer an asset for voting to a member
    """
    sp = algod_client.suggested_params()
    transfer_txn = AssetTransferTxn(sender=usiv["public"],
                                    sp=sp,
                                    receiver=member_pubk,
                                    index=idx_coin,
                                    amt=amount_for_voting)
    signed_txn = transfer_txn.sign(usiv["private"])
    txid = algod_client.send_transaction(signed_txn)
    wait_for_confirmation(algod_client,txid)

In [35]:
# asset opt-int configurations
sp                  = algod_client.suggested_params()
idx_coin            = coin_id
amount_for_voting   = int(1) # send only 1 token for voting purposes
usiv                = credentials['usiv']

In [37]:
# loop through all members to send them 1 token for voting
for i, member in enumerate(credentials["members_credentials"]):

    # retrieve member's keys
    member_pubk = credentials["members_credentials"][member]["public"]
    member_prik = credentials["members_credentials"][member]["private"]

    # if i == 6:
    #     break;

    # check that the holdings of the member are 0, otherwise continue
    if algod_client.account_info(member_pubk)["assets"]:
        if algod_client.account_info(member_pubk)["assets"][1]["amount"] != 0:
            print(f"{member} has already some holdings ({algod_client.account_info(member_pubk)['assets'][1]['amount']}), skipping")
            continue

    # transfer a coin to the asset for voting
    asset_transfer_txn(usiv, member_pubk, idx_coin, amount_for_voting=amount_for_voting)

    # get the asset amount of the member
    print(f"{member} has {algod_client.account_info(member_pubk)['assets'][1]['amount']} tokens of {algod_client.asset_info(algod_client.account_info(member_pubk)['assets'][1]['asset-id'])['params']['unit-name']}")



Current round is  41168889.
Waiting for round 41168889 to finish.
Waiting for round 41168890 to finish.
Transaction DIMWGQNGX5ZDSYXWNACAGNCPF2BHBWQQAPY5U35LD5RODR6PLESA confirmed in round 41168891.
Murphy has 1 tokens of USIV
Current round is  41168892.
Waiting for round 41168892 to finish.
Waiting for round 41168893 to finish.
Transaction AOEMEDDC5262GOZVDK5SCOQI7OB4FV3HQE4VFNSLUKDLPUL6CKNA confirmed in round 41168894.
Shaffer has 1 tokens of USIV
Beck has already some holdings (1), skipping
Current round is  41168895.
Waiting for round 41168895 to finish.
Waiting for round 41168896 to finish.
Transaction ODOYRQCQ57TXJ4OBHDIDP5DH4WRIPSO3P7AUVIPW7QXH4ZQJ3WQA confirmed in round 41168897.
Smith has 1 tokens of USIV
Current round is  41168898.
Waiting for round 41168898 to finish.
Waiting for round 41168899 to finish.
Transaction FH3Q75QEYYZAETNH5QH7FZULPX35L3W5JHGA3FR25G67BW33FF6Q confirmed in round 41168900.
Gibson has 1 tokens of USIV
Duncan has already some holdings (1), skipping
Powe